# FFI : Un exemple de A à Z

> Dans cet exemple, nous allons :
> - Linker une fonction écrite en C avec le programme Rust.
> - Partager l'allocateur Rust à la fonction écrite en C.
> - Appeler la fonction écrite en C avec en paramètre des types Rust.

## Rust bindgen

- Pour la FFI, bindgen est un outil des plus efficace. En entrée, on lui donne un fichier header .h et en sortie, il nous écrit un .rs à utiliser dans le programme :

>```
>cargo install bindgen
>```

Dans sa forme la plus simple, mono fichier, on donne un ficher header .h à bindgen. Faisons un nouveau projet nomme ffi, créer un sous-dossier c_lib et y mettre le lib.h et le merge_sort.c présent à la fin du chapitre.

- Ensuite exécuter :
>```
>cd src
>bindgen ../c_lib/lib.h > c_lib.rs
>```

- **Un fichier lib.rs sera fait à partir de lib.h :**

In [10]:
pub type size_t = ::std::os::raw::c_ulong;
#[repr(C)]
#[derive(Debug, Copy, Clone)]
pub struct custom_memory_fn {
    pub allocator:
        ::std::option::Option<unsafe extern "C" fn(arg1: size_t) -> *mut ::std::os::raw::c_void>,
    pub deallocator: ::std::option::Option<unsafe extern "C" fn(arg1: *mut ::std::os::raw::c_void)>,
}

extern "C" {                                                                                                                                      
    pub fn fusion_merge_tab(                                                                                                                      
        t1: *mut ::std::os::raw::c_void,                                                                                                          
        len: ::std::os::raw::c_int,                                                                                                               
        elmt: size_t,                                                                                                                             
        cmp: ::std::option::Option<                                                                                                               
            unsafe extern "C" fn(                                                                                                                 
                arg1: *mut ::std::os::raw::c_void,                                                                                                
                arg2: *mut ::std::os::raw::c_void,                                                                                                
            ) -> ::std::os::raw::c_int,                                                                                                           
        >,                                                                                                                                        
        mem: *mut custom_memory_fn,                                                                                                               
    ) -> *mut ::std::os::raw::c_void;                                                                                                             
}  

> Ce sont les définitions de la structure à remplir et la fonction C à appeler.

## Linkage du C avec le Rust

- Maintenant, il nous faut écrire un petit programme pour compiler le C et le linker au rust, on utilise pour cela un fichier `build.rs` que l'on place à la racine du projet :

In [9]:
use std::env;
use std::path::Path;
use std::process::Command;

fn main() {
    let out_dir = env::var("OUT_DIR").unwrap();
    // Note that there are a number of downsides to this approach, the comments
    // below detail how to improve the portability of these commands.
    Command::new("gcc")
        .args(&["c_lib/merge_sort.c", "-c", "-fPIC", "-o"])
        .arg(&format!("{}/merge_sort.o", out_dir))
        .status()
        .unwrap();
    Command::new("ar")
        .args(&["crus", "libhello.a", "merge_sort.o"])
        .current_dir(&Path::new(&out_dir))
        .status()
        .unwrap();

    println!("cargo:rustc-link-search=native={}", out_dir);
    println!("cargo:rustc-link-lib=static=hello");
}

> D'abord, il compile le fichier .c pour en faire une librairie, et il envoie les paramètres au compilateur rustc pour qu'il procède au linkage.

## La librairie est linkee a l’exécutable

> Si tout s'est bien passé, le cargo run devrait fonctionner normalement avec un simple Hello World. Intéressons-nous maintenant au code C à appeler.

- D'abord, il y a une sorte de structure qui prend un allocateur et un desallocateur en paramètre. std::alloc semble faire le boulot :

In [5]:
#[repr(C)]
#[derive(Debug, Copy, Clone)]
pub struct custom_memory_fn {
    pub allocator:
        ::std::option::Option<unsafe extern "C" fn(arg1: size_t) -> *mut ::std::os::raw::c_void>,
    pub deallocator: ::std::option::Option<unsafe extern "C" fn(arg1: *mut ::std::os::raw::c_void)>,
}

- Écrivons ces deux fonctions dans le main.rs :

```
use std::alloc::{alloc, dealloc, Layout};

unsafe extern "C" fn allocator(arg1: c_lib::size_t) -> *mut ::std::os::raw::c_void {
    let layout = Layout::array::<u64>(arg1 as usize / std::mem::size_of::<u64>());
    let ptr = alloc(layout.unwrap());
    ptr as *mut ::std::os::raw::c_void
}

unsafe extern "C" fn deallocator(arg1: *mut ::std::os::raw::c_void) {
    let layout = Layout::new::<[u64; TAB_SIZE]>();
    dealloc(arg1 as *mut u8, layout);
}
```

## La fonction de comparaison

- Pareil, apparemment, on doit envoyer l’adresse d'une fonction de comparaison. Ecrivons-la :

In [7]:
unsafe extern "C" fn cmp(
    arg1: *mut std::os::raw::c_void,
    arg2: *mut std::os::raw::c_void,
) -> std::os::raw::c_int {
    if *(arg1 as *mut u64) < *(arg2 as *mut u64) {
        0
    } else {
        1
    }
}

> Tout est a peu près unsafe la !

## La fonction main du programme

- Et enfin, voila comment completer la fonction main du programme :

```
const TAB_SIZE: usize = 1024 * 2;

fn main() {
    let mut custom = c_lib::custom_memory_fn {
        allocator: Some(allocator),
        deallocator: Some(deallocator),
    };

    let layout = Layout::array::<u64>(TAB_SIZE);
    let v = unsafe {
        let ptr = alloc(layout.unwrap());
        let mut v = Vec::<u64>::from_raw_parts(ptr as *mut u64, TAB_SIZE, TAB_SIZE);
        // remplit le vecteur avec des valeurs decroissantes
        for i in 0..(TAB_SIZE) {
            v[i] = ((TAB_SIZE) - i) as u64;
        }
        let old_ptr = v.as_ptr();
        let ptr = c_lib::fusion_merge_tab(
            v.as_ptr() as *mut ::std::os::raw::c_void,
            TAB_SIZE as i32,
            std::mem::size_of::<u64>() as u64,
            Some(cmp),
            &mut custom as *mut c_lib::custom_memory_fn as *mut c_lib::custom_memory_fn,
        );

        println!("Finito !");
        dbg!(old_ptr);
        dbg!(ptr);
        if old_ptr != ptr as *const u64 {
            v.leak(); // !!!
            v = Vec::<u64>::from_raw_parts(ptr as *mut u64, TAB_SIZE, TAB_SIZE);
        }
        v
    };
    for i in v.into_iter() {
        dbg!(i);
    }
}
```

**lib.h**
```
#ifndef LIB_H
# define LIB_H

typedef unsigned long int size_t;

struct custom_memory_fn {
    void *(*allocator)(size_t);
    void (*deallocator)(void *);
};

void *fusion_merge_tab(void *t1,
                       int len,
                       size_t elmt,
                       int (*cmp)(void *, void *),
                       struct custom_memory_fn *mem);

#endif
```

**merge_sort.c**
```
#include "lib.h"

#include <stdbool.h>
#include <errno.h>
#include <string.h>

struct	s_info {
    int offset;
    int (*cmp)(void *, void *);
    size_t elmt_len;
};

static void merge_mod(char *s1, char *s2, char *end, struct s_info *w) {
    char *p_gr_1;
    char *p_gr_2;

    while ((p_gr_1 = s1) < end) {
        p_gr_2 = p_gr_1 + (w->offset * w->elmt_len);
        while (true) {
			if (p_gr_2 < end) {
                if (w->cmp((void *)p_gr_2, (void *)p_gr_1) == 0) {
                    memcpy(s2, p_gr_2, w->elmt_len);
                    p_gr_2 += w->elmt_len;
                } else {
                    memcpy(s2, p_gr_1, w->elmt_len);
                    p_gr_1 += w->elmt_len;
                }
                s2 += w->elmt_len;
            }
			if (p_gr_1 == (s1 + w->offset * w->elmt_len)) {
				while (p_gr_2 != (s1 + (2 * w->offset * w->elmt_len)) && p_gr_2 < end) {
                    memcpy(s2, p_gr_2, w->elmt_len);
                    s2 += w->elmt_len;
                    p_gr_2 += w->elmt_len;
                }
				break ;
			}            
            else if (p_gr_2 == (s1 + (2 * w->offset * w->elmt_len)) || p_gr_2 >= end) {
                while (p_gr_1 != (s1 + w->offset * w->elmt_len) && p_gr_1 < end) {
                    memcpy(s2, p_gr_1, w->elmt_len);
                    s2 += w->elmt_len;
                    p_gr_1 += w->elmt_len;
                }
                break ;
            }
        }
        s1 += 2 * w->offset * w->elmt_len;
    }
}

static void *exec(char *t1,
                  char *t2,
                  int elmt_nb,
                  size_t elmt_len,
                  int (*cmp)(void *, void *)) {
    struct s_info   w;
    int             state;

    w.cmp = cmp;
    w.offset = 1;
    w.elmt_len = elmt_len;
    state = false;

    while (w.offset < elmt_nb) {
        if (state == false)
            merge_mod((char *)t1, (char *)t2, (char *)t1 + (elmt_nb * elmt_len), &w);
        else
            merge_mod((char *)t2, (char *)t1, (char *)t2 + (elmt_nb * elmt_len), &w);
        state = (state) ? false : true;
        w.offset <<= 1;
    }
    return ((state) ? t2 : t1);
}

void *fusion_merge_tab(void *t1,
                       int elmt_nb,
                       size_t elmt_len,
                       int (*cmp)(void *, void *),
                       struct custom_memory_fn *mem) {
    void *t2;
    void *tmp;
 
    if (!cmp || !mem || !mem->allocator || !mem->deallocator || !t1)
        return (NULL);
    if (!elmt_nb)
        return (0);
    if (!(t2 = (void *)mem->allocator(elmt_nb * elmt_len)))
        return NULL;
    if ((tmp = exec(t1, t2, elmt_nb, elmt_len, cmp)) == t1) {
        mem->deallocator(t2);
        return t1;
    } else {
        mem->deallocator(t1);
        return tmp;
    }
}

/*
 * #include <stdio.h>
 * 
 * int cmp(void *a, void *b) {
 *     if (*(int *)a >= *(int *)b) return 1;
 *     else return 0;
 * }
 * 
 * int main() {
 *     struct custom_memory_fn custom;
 *     custom.allocator = malloc;
 *     custom.deallocator = free;
 * 
 *     int *tab = (int *)custom.allocator(sizeof(int) * 6);
 * 
 *     tab[0] = 42;
 *     tab[1] = 10;
 *     tab[2] = -3;
 *     tab[3] = -43;
 *     tab[4] = 90;
 *     tab[5] = 2;
 *     tab = fusion_merge_tab((void*)tab, 6, sizeof(int), cmp, &custom);
 *     for (int i = 0; i < 6; i++) {
 *         printf("%i\n", i);
 *         printf("%i\n", tab[i]);
 *     }
 *     custom.deallocator(tab);
 *     return 0;
 * }
 */

```